In [7]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd
import csv

In [8]:
df = pd.read_csv('graph_data.csv')
display(df)

n    m     Edges
0    10   10    (1, 9)
1    10   10    (1, 3)
2    10   10    (2, 5)
3    10   10    (3, 5)
4    10   10    (4, 8)
..   ..  ...       ...
995  20  180  (17, 20)
996  20  180  (17, 18)
997  20  180  (18, 19)
998  20  180  (18, 20)
999  20  180  (19, 20)

[1000 rows x 3 columns]

In [9]:
graph_data = {}
for index, row in df.iterrows():
    n = row['n']
    m = row['m']
    edge_tuple = tuple(map(int, row['Edges'].strip("()").split(",")))

    if n not in graph_data:
        graph_data[n] = {}

    if m not in graph_data[n]:
        graph_data[n][m] = []

    graph_data[n][m].append(edge_tuple)

In [10]:
def lp_rounding(n, edges, weights=None):

    if weights is None:
        weights = [1] * n
    c = np.array(weights)
    A = []
    b = []
    for u, v in edges:
        constraint = np.zeros(n)
        # Adjusting indices to be 0-based
        constraint[u-1] = -1
        constraint[v-1] = -1
        A.append(constraint)
        b.append(-1)
    A = np.array(A)
    b = np.array(b)
    bounds = [(0, 1) for _ in range(n)]
    res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')
    x_star = res.x
    cover = [i for i in range(n) if x_star[i] >= 0.5]
    return cover, x_star

In [11]:
results = {}

for n in graph_data:
    results[n] = {}
    for m in graph_data[n]:
        edges = graph_data[n][m]

        cover, x = lp_rounding(n, edges)

        # Calculate LP optimal (sum of fractional values)
        lp_optimal = np.sum(x)

        approximation_cover_size = len(cover)

        results[n][m] = {
            'lp_optimal': lp_optimal,
            'approximation_cover_size': approximation_cover_size
        }

# Display the results
display(results)

{10: {10: {'lp_optimal': 4.0, 'approximation_cover_size': 4},
  20: {'lp_optimal': 5.0, 'approximation_cover_size': 10},
  30: {'lp_optimal': 5.0, 'approximation_cover_size': 10},
  40: {'lp_optimal': 5.0, 'approximation_cover_size': 10}},
 20: {20: {'lp_optimal': 8.0, 'approximation_cover_size': 8},
  40: {'lp_optimal': 10.0, 'approximation_cover_size': 20},
  60: {'lp_optimal': 10.0, 'approximation_cover_size': 20},
  80: {'lp_optimal': 10.0, 'approximation_cover_size': 20},
  100: {'lp_optimal': 10.0, 'approximation_cover_size': 20},
  120: {'lp_optimal': 10.0, 'approximation_cover_size': 20},
  140: {'lp_optimal': 10.0, 'approximation_cover_size': 20},
  160: {'lp_optimal': 10.0, 'approximation_cover_size': 20},
  180: {'lp_optimal': 10.0, 'approximation_cover_size': 20}}}

In [12]:
approx_results = pd.read_csv('results_data.csv')
lp_results_list = []
for n in results:
    for m in results[n]:
        lp_results_list.append({'n': n, 'm': m, 'lp_optimal': results[n][m]['lp_optimal']})
lp_results_df = pd.DataFrame(lp_results_list)


comparison_df = pd.merge(approx_results, lp_results_df, on=['n', 'm'])

# Approximation Factor = Size of Approximate Cover / LP Optimal
comparison_df['2 Factor Approx Factor (vs LP Optimal)'] = comparison_df['2 Factor Approximation'] / comparison_df['lp_optimal']

display(comparison_df)

n    m  Brute Force  2 Factor Approximation  lp_optimal  \
0   10   10            4                       8         4.0   
1   10   20            6                      10         5.0   
2   10   30            7                      10         5.0   
3   10   40            8                      10         5.0   
4   20   20            8                      14         8.0   
5   20   40           12                      16        10.0   
6   20   60           13                      18        10.0   
7   20   80           14                      18        10.0   
8   20  100           15                      20        10.0   
9   20  120           16                      20        10.0   
10  20  140           16                      18        10.0   
11  20  160           17                      20        10.0   
12  20  180           18                      20        10.0   

    2 Factor Approx Factor (vs LP Optimal)  
0                                     2.00  
1                                     2.00  
2                                     2.00  
3                                     2.00  
4                                     1.75  
5                                     1.60  
6                                     1.80  
7                                     1.80  
8                                     2.00  
9                                     2.00  
10                                    1.80  
11                                    2.00  
12                                    2.00

In [13]:
comparison_df.to_csv('comparison_results.csv', index=False)